<a href="https://colab.research.google.com/github/chi-hun/chi-hun/blob/main/downcast_and_Parquet_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# module

In [ ]:
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
x_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘/제주도 도로 교통량 예측/x_train_2.csv')

# downcast (파일타입을 변경 메모리 사용량을 줄인다)

In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 30 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   base_hour                  float64
 1   road_in_use                int64  
 2   multi_linked               int64  
 3   connect_code               int64  
 4   road_type                  int64  
 5   start_turn_restricted      int64  
 6   end_turn_restricted        int64  
 7   month                      int64  
 8   lane_count_1               int64  
 9   lane_count_2               int64  
 10  lane_count_3               int64  
 11  road_rating_103            int64  
 12  road_rating_106            int64  
 13  road_rating_107            int64  
 14  maximum_speed_limit_30.0   int64  
 15  maximum_speed_limit_40.0   int64  
 16  maximum_speed_limit_50.0   int64  
 17  maximum_speed_limit_60.0   int64  
 18  maximum_speed_limit_70.0   int64  
 19  maximum_speed_limit_80.0   int64  
 20  we

In [ ]:
x_train.memory_usage()

Index                             128
base_hour                    37609736
road_in_use                  37609736
multi_linked                 37609736
connect_code                 37609736
road_type                    37609736
start_turn_restricted        37609736
end_turn_restricted          37609736
month                        37609736
lane_count_1                 37609736
lane_count_2                 37609736
lane_count_3                 37609736
road_rating_103              37609736
road_rating_106              37609736
road_rating_107              37609736
maximum_speed_limit_30.0     37609736
maximum_speed_limit_40.0     37609736
maximum_speed_limit_50.0     37609736
maximum_speed_limit_60.0     37609736
maximum_speed_limit_70.0     37609736
maximum_speed_limit_80.0     37609736
weight_restricted_0.0        37609736
weight_restricted_32400.0    37609736
weight_restricted_43200.0    37609736
weight_restricted_50000.0    37609736
road_name_                   37609736
road_name_교 

In [ ]:
def downcast(df, vervose = True):
    if vervose:
        mem_st = round(df.memory_usage().sum() / 1024**2, 2)
    for i in df.columns:
        columns_dtype = df[i].dtype.name
        if columns_dtype == 'object': # object 타입은 그대로 사용
            pass
        elif columns_dtype == 'bool': # bool은 int8로 변경
            df[i] = df[i].astype('int8')
        elif columns_dtype.startswith('int') or (df[i].round() == df[i]).all(): # int로 시작하거나 반올림한 값이 같으면 정수 downcast
            df[i] = pd.to_numeric(df[i], downcast='integer')
        else:
            df[i] = pd.to_numeric(df[i], downcast='float') # float downcast
    if vervose:
        mem_la = round(df.memory_usage().sum() / 1024**2, 2)
        zi = round(((mem_st - mem_la) / mem_st) * 100, 2)
        print(f'{mem_st} -> {mem_la} 압출율 : {zi}%')
    return df

In [ ]:
x_train = downcast(x_train)

1076.02 -> 161.4 압출율 : 85.0%


In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 30 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   base_hour                  float32
 1   road_in_use                int8   
 2   multi_linked               int8   
 3   connect_code               int8   
 4   road_type                  int8   
 5   start_turn_restricted      int8   
 6   end_turn_restricted        int8   
 7   month                      int8   
 8   lane_count_1               int8   
 9   lane_count_2               int8   
 10  lane_count_3               int8   
 11  road_rating_103            int8   
 12  road_rating_106            int8   
 13  road_rating_107            int8   
 14  maximum_speed_limit_30.0   int8   
 15  maximum_speed_limit_40.0   int8   
 16  maximum_speed_limit_50.0   int8   
 17  maximum_speed_limit_60.0   int8   
 18  maximum_speed_limit_70.0   int8   
 19  maximum_speed_limit_80.0   int8   
 20  we

In [ ]:
x_train.memory_usage()

Index                             128
base_hour                    18804868
road_in_use                   4701217
multi_linked                  4701217
connect_code                  4701217
road_type                     4701217
start_turn_restricted         4701217
end_turn_restricted           4701217
month                         4701217
lane_count_1                  4701217
lane_count_2                  4701217
lane_count_3                  4701217
road_rating_103               4701217
road_rating_106               4701217
road_rating_107               4701217
maximum_speed_limit_30.0      4701217
maximum_speed_limit_40.0      4701217
maximum_speed_limit_50.0      4701217
maximum_speed_limit_60.0      4701217
maximum_speed_limit_70.0      4701217
maximum_speed_limit_80.0      4701217
weight_restricted_0.0         4701217
weight_restricted_32400.0     4701217
weight_restricted_43200.0     4701217
weight_restricted_50000.0     4701217
road_name_                    4701217
road_name_교 

# parquet (파일종류를 변경, 파일사이즈를 줄인다)

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/데이콘/제주도 도로 교통량 예측/x_train_2.csv'
size_st = round(os.path.getsize(path) / (1024 *1024), 2)
df = pd.read_csv(path)
df.to_parquet('train.parquet')
path_r = '/content/train.parquet'
size_la = round(os.path.getsize(path_r) / (1024 *1024), 2)
df = pd.read_parquet(path_r)
zi = round(((size_st - size_la) / size_st) * 100, 2)
print(f'{size_st}MB -> {size_la}MB 압축율 : {zi}%')

343.05MB -> 16.48MB 압축율 : 95.2%


In [ ]:
#pip install fastparquet # 에러1시 실행

# apt update && apt install -y build-essential #에러2시 실행
# pip install fastparquet

In [ ]:
df.to_parquet('sample.parquet', compression='gzip') # 기본 snappy 압축에서 gzip 압축을 사용하고 싶으면